In [ ]:
import utils_BudykoSellers
import utils_emulator
import numpy as np

%load_ext autoreload
%autoreload 2

## Experiment 3: Noisy Three Box (noise)

This notebook runs the noisy three box model for the following scenarios and emulation techniques:

Scenarios:
1. Abrupt - An abrupt doubling of CO2 concentration; corresponds roughly to the *Abrupt2xCO2* CMIP experiment.
2. High Emissions - An exponential increase of CO2 concentration in time; corresponds roughly to *SSP585*.
3. Plateau - An increase in CO2 concentration in time that follows a hyperbolic tangent, increasing exponentially and then tapering off; corresponds roughly to *SSP245*.
4. Overshoot - An increase in CO2 concentration in time that follows a Gaussian profile, increasing and decreasingly rapidly; inspired by *SSP119*, but decreases more quickly.

Emulators:
1. Pattern Scaling - Time-invariant pattern based on linear regression from global mean temperature to local temperature.
2. Fluctuation Dissipation Theorem - Response functions derived through perturbation experiment.
3. Deconvolution - Response functions solved for from any general experiment.
4. Modal Fitting - Response functions fit from any general experiment.
5. Dynamic Mode Decomposition (DMD) - Approximating system dynamics with a linear operator.
6. Extended DMD - Approximating system dynamics with nonlinear basis functions.

### Setup and Run Scenarios

Required before creating/evaluating emulators.

In [ ]:
# Required variables
t_end = 251
t = np.arange(0,t_end)
n_boxes = 3

# Ensemble parameters
n_ensemble = 50   # Ensemble members
xi         = 0.15 # Noise strength

# Initialize forcing vectors and run scenarios
scenarios = ['Abrupt','High Emissions','Plateau','Overshoot']
full_outputs_ensemble, forcings_ensemble, T_out_ensemble = {}, {}, {}
full_outputs_single, forcings_single, T_out_single = {}, {}, {}
for i, scen in enumerate(scenarios):
  full_outputs_ensemble[scen], forcings_ensemble[scen], T_out_ensemble[scen] = [], [] ,[]

  # Run deterministic scenarios once
  full_outputs_single[scen] = utils_BudykoSellers.Run_Budyko_Sellers(scen_flag=i, n_boxes=n_boxes, diff_flag=1)
  forcings_single[scen] = np.tile(full_outputs_single[scen]['forcing_ts'], (n_boxes, 1))
  T_out_single[scen] = np.squeeze(full_outputs_single[scen]['T_ts'])[0:n_boxes,:]

  # Iterate over all ensemble members
  for n in range(n_ensemble):
    full_outputs_ensemble[scen].append(utils_BudykoSellers.Run_Budyko_Sellers(scen_flag=i, xi=xi, n_boxes=n_boxes, diff_flag=1))
    forcings_ensemble[scen].append(np.tile(full_outputs_ensemble[scen][-1]['forcing_ts'], (n_boxes, 1)))
    T_out_ensemble[scen].append(np.squeeze(full_outputs_ensemble[scen][-1]['T_ts'])[0:n_boxes,:])


### Take mean over the entire ensemble (for calculations later)

In [ ]:
mean_forcing, mean_T = {}, {}
for scen in scenarios:
  mean_forcing[scen] = np.mean(np.stack(forcings_ensemble[scen], axis=0), axis=0)
  mean_T[scen] = np.mean(np.stack(T_out_ensemble[scen], axis=0), axis=0)

### Method I: Pattern Scaling

#### Baseline performance

In [ ]:
verbose    = False  # Show error output
plot       = False  # Plot emulator performance
save_error = False  # Save error output

operator_PS, T_pred_PS, error_metrics_PS = utils_emulator.emulate_scenarios('PS', scenarios=scenarios, outputs=T_out_single, forcings=forcings_single, verbose=verbose)
NRMSE_base_PS = utils_emulator.calc_base_NRMSE(error_metrics_PS, scenarios)

if plot:
  utils_emulator.plot_true_pred(T_out_single, T_pred_PS, scenarios, operator='PS')

if save_error:
  utils_emulator.save_results(error_metrics_PS, 'exp3_I_PS_error_single')

#### Noisy performance

In [ ]:
save_error = False

n_choices = 10 # Number of subsamples to take
NRMSE_all_PS = utils_emulator.evaluate_ensemble('PS', T_out_ensemble, mean_T, forcings_ensemble, mean_forcing, scenarios, n_ensemble, n_choices)

if save_error:
  utils_emulator.save_results(NRMSE_all_PS, 'exp3_I_PS_error_ensemble')

### Method II: Fluctuation Dissipation Theorem

Note: Since the noise here is added linearly, using the FDT over the entire ensemble leads to the same response function as in the baseline case. We therefore test this baseline against the noisy ensemble.

#### Baseline performance

In [ ]:
verbose    = False    # Show error output
plot       = False    # Plot emulator performance
save_error = False    # Save error output

dt = 1 # Timestep (year)
operator_FDT, T_pred_FDT, error_metrics_FDT = utils_emulator.emulate_scenarios('FDT', scenarios=scenarios, outputs=T_out_single, forcings=forcings_single, n_boxes=n_boxes, dt=dt, diff_flag=1, delta=1, verbose=verbose)

if plot:
  utils_emulator.plot_true_pred_FDT(T_out_single, T_pred_FDT, scenarios)

if save_error:
  utils_emulator.save_results(error_metrics_FDT, 'exp3_II_FDT_error_single')

#### Noisy Performance

In [ ]:
save_error = False

n_choices = 10 # Number of subsamples to take
dt        = 1  # Timestep (years)
NRMSE_all_FDT = utils_emulator.evaluate_ensemble('FDT', T_out_ensemble, mean_T, forcings_ensemble, mean_forcing, scenarios, n_ensemble, n_choices, dt=dt, G_FDT=operator_FDT)

if save_error:
  utils_emulator.save_results(NRMSE_all_FDT, 'exp3_II_FDT_error_ensemble')

### Method III: Deconvolution

#### Baseline performance

In [ ]:
verbose    = False   # Show error output
plot       = False   # Plot emulator performance
save_error = False   # Save error output

dt = 1 # Timestep (year)
operator_deconvolve, T_pred_deconvolve, error_metrics_deconvolve = utils_emulator.emulate_scenarios('deconvolve', scenarios=scenarios, outputs=T_out_single, forcings=forcings_single, dt=dt, regularize=True, verbose=verbose)
NRMSE_base_deconvolve = utils_emulator.calc_base_NRMSE(error_metrics_deconvolve, scenarios)

if plot:
  utils_emulator.plot_true_pred(T_out_single, T_pred_deconvolve, scenarios, operator='deconvolve')

if save_error:
  utils_emulator.save_results(error_metrics_deconvolve, 'exp3_III_deconv_error_single')

#### Noisy performance

In [ ]:
save_error = False

n_choices = 10 # Number of subsamples to take
dt        = 1  # Timestep (years)
NRMSE_all_deconvolve = utils_emulator.evaluate_ensemble('deconvolve', T_out_ensemble, mean_T, forcings_ensemble, mean_forcing, scenarios, n_ensemble, n_choices, dt=dt)

if save_error:
  utils_emulator.save_results(NRMSE_all_deconvolve, 'exp3_III_deconv_error_ensemble')

### Method IV: Modal Fitting

#### Baseline performance

In [ ]:
verbose    = False   # Show error output and optimal parameters
plot       = False   # Plot emulator performance
save_error = False   # Save error output

dt = 1 # Timestep (year)
operator_modal, T_pred_modal, error_metrics_modal = utils_emulator.emulate_scenarios('modal', scenarios=scenarios, outputs=T_out_single, forcings=forcings_single, t=t, dt=dt, n_boxes=n_boxes, n_modes=3, verbose=verbose)
NRMSE_base_modal = utils_emulator.calc_base_NRMSE(error_metrics_modal, scenarios)

if plot:
  utils_emulator.plot_true_pred(T_out_single, T_pred_modal, scenarios, operator='modal')

if save_error:
  utils_emulator.save_results(error_metrics_modal, 'exp3_IV_modal_error_single')

#### Noisy performance

In [ ]:
save_error = False

n_choices = 10 # Number of subsamples to take
dt        = 1  # Timestep (years)
NRMSE_all_modal = utils_emulator.evaluate_ensemble('modal', T_out_ensemble, mean_T, forcings_ensemble, mean_forcing, scenarios, n_ensemble, n_choices, t=t, dt=dt, n_boxes=n_boxes, n_modes=3)

if save_error:
  utils_emulator.save_results(NRMSE_all_modal, 'exp3_IV_modal_error_ensemble')

### Method V: Dynamic Mode Decomposition (DMD)

#### Baseline performance

In [ ]:
verbose    = False   # Show error output
plot       = False   # Plot emulator performance
save_error = False   # Save error output

n_steps = len(t)            # No. timesteps
w0      = np.zeros(n_boxes) # Initial condition
operator_DMD, T_pred_DMD, error_metrics_DMD = utils_emulator.emulate_scenarios('DMD', scenarios=scenarios, outputs=T_out_single, forcings=forcings_single, w0=w0, t=t, n_steps=n_steps, n_boxes=n_boxes, verbose=verbose, regularize=True)
NRMSE_base_DMD = utils_emulator.calc_base_NRMSE(error_metrics_DMD, scenarios)

if plot:
  utils_emulator.plot_true_pred(T_out_single, T_pred_DMD, scenarios, operator='DMD')

if save_error:
  utils_emulator.save_results(error_metrics_DMD, 'exp3_V_DMD_error_single')

#### Noisy performance

In [ ]:
save_error = False

n_choices = 10                # Number of subsamples to take
n_steps   = len(t)            # No. timesteps
w0        = np.zeros(n_boxes) # Initial condition
NRMSE_all_DMD = utils_emulator.evaluate_ensemble('DMD', T_out_ensemble, mean_T, forcings_ensemble, mean_forcing, scenarios, n_ensemble, n_choices, w0=w0, n_steps=n_steps)

if save_error:
  utils_emulator.save_results(NRMSE_all_DMD, 'exp3_V_DMD_error_ensemble')

### Method VI: Extended DMD

#### Baseline performance

In [ ]:
verbose    = False   # Show error output
plot       = False   # Plot emulator performance
save_error = False   # Save error output

n_steps = len(t)            # No. timesteps
w0      = np.zeros(n_boxes) # Initial condition

# Basis functions
w_dict = utils_emulator.Vector_Dict(method='hermite', degree=1)
F_dict = utils_emulator.Vector_Dict(method='hermite', degree=1)
operator_EDMD, T_pred_EDMD, error_metrics_EDMD = utils_emulator.emulate_scenarios('EDMD', scenarios=scenarios, outputs=T_out_single, forcings=forcings_single, w0=w0, t=t, n_steps=n_steps, n_boxes=n_boxes, w_dict=w_dict, F_dict=F_dict, verbose=verbose)
NRMSE_base_EDMD = utils_emulator.calc_base_NRMSE(error_metrics_EDMD, scenarios)

if plot:
  utils_emulator.plot_true_pred(T_out_single, T_pred_EDMD, scenarios, operator='EDMD')

if save_error:
  utils_emulator.save_results(error_metrics_EDMD, 'exp3_VI_EDMD_error_single')

#### Noisy performance

In [ ]:
save_error = False

n_choices = 10                # Number of subsamples to take
n_steps   = len(t)            # No. timesteps
w0        = np.zeros(n_boxes) # Initial condition

# Basis functions
w_dict = utils_emulator.Vector_Dict(method='hermite', degree=1)
F_dict = utils_emulator.Vector_Dict(method='hermite', degree=1)
NRMSE_all_EDMD = utils_emulator.evaluate_ensemble('EDMD', T_out_ensemble, mean_T, forcings_ensemble, mean_forcing, scenarios, n_ensemble, n_choices, w0=w0, n_steps=n_steps, n_boxes=n_boxes, w_dict=w_dict, F_dict=F_dict)

if save_error:
  utils_emulator.save_results(NRMSE_all_EDMD, 'exp3_VI_EDMD_error_ensemble')